In [19]:

from jetbot import Robot
import traitlets
import ipywidgets.widgets as widgets
from jetbot import Camera
from jetbot import bgr8_to_jpeg
import cv2
import numpy as np
import time



Thresholding is based on lab environment whee the tape was on grey background.

In [20]:
# Initialize camera
camera = Camera.instance()
image = widgets.Image(format='jpeg', width=640, height=480)
Blackline_image = widgets.Image(format='jpeg', width=640, height=480)

def process_frame(change): 
    frame = change['new'] 
    roi=frame[50:150, 0:640] 
    Blackline=cv2.inRange(roi, (15,15,15), (100,100,100)) 
    kernel=np.ones((5,5), np.uint8) 
    Blackline=cv2.erode(Blackline,kernel,iterations=5) 
    Blackline=cv2.dilate(Blackline,kernel,iterations=6) 
    contours,hierarchy = cv2.findContours(Blackline.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE) 
    # cv2.drawContours(roi,contours,-1,(0,255,0),1) 
    if len(contours)>0: 
        largest = max(contours, key=cv2.contourArea)
        x,y,w,h=cv2.boundingRect(largest) 
      #  cv2.rectangle(roi, (x,y), (x+w,y+h),(0,0,255),1) 
        cv2.line(roi, (x + (w//2), 50), (x + (w//2), 150), (255, 0, 0), 1) 
    image.value = bgr8_to_jpeg(frame) 
    Blackline_bgr=cv2.cvtColor(Blackline, cv2.COLOR_GRAY2BGR) 
    Blackline_image.value=bgr8_to_jpeg(Blackline_bgr) 

display(image) 
camera.observe(process_frame, names='value')

Image(value=b'', format='jpeg', height='480', width='640')

In [21]:
camera.stop()